Tools, helpful functions for development and debugging
====================



In [1]:
from sage.algebras.flag_algebras import *

In [72]:
sys.setprofile(None)

def tracefunc(frame, event, arg, indent=[0]):
    if event == "call":
        indent[0] += 2
        module_name = inspect.getmodule(frame.f_code).__name__ if inspect.getmodule(frame.f_code) else "unknown"
        class_name = ""
        try:
            class_name = inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True)[-1][0][0].__name__
        except:
            if 'self' in frame.f_locals:
                class_name = str(inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True))
            else:
                class_name = ""
        function_name = frame.f_code.co_name
        tst = module_name + function_name
        if "IPython" in tst or "enum" in tst or "typing" in tst or "traitlets" in tst or "util" in tst or "ipykernel" in tst:
            pass
        else:
            if "unknown" in tst or "tokenize" in tst or "logging" in tst or "json" in tst or "preparse" in tst or indent[0]<=0:
                pass
            else:
                print("-" * indent[0] + "> call function", function_name, "in", class_name, "from", module_name)
    elif event == "return":
        indent[0] -= 2
    return tracefunc

#sys.setprofile(tracefunc)

#asd = sum([G(0)])

In [3]:
#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#for debug printing values with their variable names
def debug(*args):
    import inspect
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = s.split('(')[1].split(')')[0].split(',')
    names = [name.strip() for name in r]
    
    for name, value in zip(names, args):
        if isinstance(value, str):
            print(value)
        else:
            print(f"{name}: {value}")


In [14]:
def verify_solution(self, solution, target_element, target_size, maximize=True, positives=None, construction=None):
    #
    # Checking eigenvalues and positivity constraints
    #
    
    if len(solution)==2 and solution[0] in QQ:
        solution = solution[1]
    if len(solution)==3 and solution[0] in QQ:
        solution = solution[1]
    
    if len(solution[-1])>0 and min(solution[-1])<0:
        print("Solution is not valid!")
        print("Linear constraint's coefficient is negative {}".format(min(solution[-1])))
        return -1

    for ii,X in enumerate(solution[:-1]):
        if min(X.eigenvalues())<0:
            print("Solution is not valid!")
            print("Matrix {} is not semidefinite: {}".format(ii, min(X.eigenvalues())))
            return -1
    
    print("Solution matrices are all semidefinite, linear coefficients are all non-negative")

    #
    # Initial setup
    #
    
    mult = -1 if maximize else 1
    base_flags = self.generate_flags(target_size)
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    if target_element.ftype().size()==0:
        one_vector = vector([1]*len(base_flags))
    else:
        one_vector = (target_element.ftype().project()<<(target_size - target_element.ftype().size())).values()
    
    #
    # Create the types used in the calculation
    #
    
    ftype_data = []
    for fs in range(target_size-2, 1, -2):
        for fl in self.generate_flags(fs):
            ftype = fl.subflag(ftype_points=list(range(fs)))
            ns = (target_size + fs)/2
            ftype_data.append((ns, ftype, target_size))
    ftype_data.sort()
    
    print("Done calculating types relevant for the calculation")
    
    #
    # Create the semidefinite matrix data
    #
    
    table_list = []
    for ii, dat in enumerate(ftype_data):
        ns, ftype, target_size = dat
        #calculate the table
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        if table!=None:
            table_list.append(table)
        print("Done with mult table for {}".format(ftype))

    print("Done calculating semidefinite constraints")
    
    #
    # Create the data from linear constraints
    #

    positives_list_exact = []
    if positives != None:
        for ii, fv in enumerate(positives):
            if isinstance(fv, Flag):
                continue
            nf = fv.size()
            df = target_size + fv.ftype().size() - nf
            mult_table = self.mul_project_table(nf, df, fv.ftype(), ftype_inj=[], target_size=target_size)
            fvvals = fv.values()
            m = matrix(QQ, [vector(fvvals*mat) for mat in mult_table])
            positives_list_exact += list(m.T)
            print("Done with positivity constraint {}".format(ii))
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), len(base_flags), positives_list_exact)
    
    print("Done calculating linear constraints")

    #
    # Calculate the bound the solution provides
    #
    
    slacks = target_vector_exact - positives_matrix_exact.T*solution[-1]
    for ii, table in enumerate(table_list):
        for gg, mat_gg in enumerate(table):
            slacks[gg] -= sum([mat_gg.rows()[jj]*solution[ii][jj] for jj in range(mat_gg.nrows())])
    res = min(slacks)*mult
    
    print("The solution is valid, it proves the bound {}".format(res))
    
    return res

In [15]:
verify_solution(G, res, G(2), 4)

Solution matrices are all semidefinite, linear coefficients are all non-negative
Done calculating types relevant for the calculation
Done with mult table for Ftype on 2 points with edges=[]
Done with mult table for Ftype on 2 points with edges=[[0, 1]]
Done calculating semidefinite constraints
Done calculating linear constraints
The solution is valid, it proves the bound 2/3


2/3

In [1]:
G = GraphTheory
G.exclude(G(4))
res = G.optimize_problem(G(2), 4, exact=True, certificate=True)

Base flags generated, their number is 10
The relevant ftypes are constructed, their number is 2
Block sizes before symmetric/asymmetric change is applied: [4, 4]


Done with mult table for Ftype on 2 points with edges=[[0, 1]]: : 2it [00:00, 295.33it/s]

Tables finished
Constraints finished
Running sdp without construction. Used block sizes are [3, 1, 3, 1, -10, -2]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 


Iter:  1 Ap: 1.00e+00 Pobj: -1.9624955e+01 Ad: 7.44e-01 Dobj: -7.2177272e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.9528056e+01 Ad: 9.47e-01 Dobj: -4.2455018e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.1538698e+01 Ad: 8.63e-01 Dobj: -4.2284259e-01 
Iter:  4 Ap: 9.68e-01 Pobj: -3.0877689e+00 Ad: 8.33e-01 Dobj: -4.2894578e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.0177635e+00 Ad: 8.54e-01 Dobj: -4.5853098e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -7.8932054e-01 Ad: 7.97e-01 Dobj: -5.5689303e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -7.0457893e-01 Ad: 7.53e-01 Dobj: -6.1303953e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -6.7790466e-01 Ad: 8.00e-01 Dobj: -6.4695184e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -6.6808323e-01 Ad: 9.22e-01 Dobj: -6.6308132e-01 
Iter: 10 Ap: 1.00e+00 Pobj: -6.6677694e-01 Ad: 1.00e+00 Dobj: -6.6645562e-01 
Iter: 11 Ap: 9.89e-01 Pobj: -6.6667287e-01 Ad: 1.00e+00 Dobj: -6.6665842e-01 
Iter: 12 Ap: 1.00e+00 Pobj: -6.6666728e-01 Ad: 1.00e+00 Dobj: -6.6666666e-01 
Iter: 13 Ap: 1.00e+00 Pobj: -6.6666670e-01 Ad: 9.74e-01 Dobj: -6

In [43]:
Xo = [matrix(res['X'][0]), matrix(res['X'][1])]

In [29]:
fr = [
    matrix([[7/6, -329/512], [-329/512, 885/1024]]),
    matrix([[3/8]]),
    matrix([[1/6, 29/1024], [29/1024, 143/1024]]),
    matrix([[285/512]])
]
bc = [
    matrix([[1, -1/2, -1/2, 0], [0, 0, 0, 1]]),
    matrix([[0, 1, -1, 0]]),
    matrix([[1, 0, 0, -2], [0, 1, 1, 0]]),
    matrix([[0, 1, -1, 0]])
]

In [49]:
Xr = [bc[0].T * fr[0] * bc[0] + bc[1].T * fr[1] * bc[1], 
     bc[2].T * fr[2] * bc[2] + bc[3].T * fr[3] * bc[3]]
print(Xo[1])
print()
print(Xr[1].n())

[   0.1666664674330672    0.0229558138755806  0.022955813875580638  -0.33333313001851994]
[   0.0229558138755806    0.6136578762723662   -0.3535733181306354  -0.04591120962709709]
[ 0.022955813875580638   -0.3535733181306354    0.6136578762723653 -0.045911209627097195]
[ -0.33333313001851994  -0.04591120962709709 -0.045911209627097195    0.6666666595244964]

[  0.166666666666667  0.0283203125000000  0.0283203125000000  -0.333333333333333]
[ 0.0283203125000000   0.696289062500000  -0.416992187500000 -0.0566406250000000]
[ 0.0283203125000000  -0.416992187500000   0.696289062500000 -0.0566406250000000]
[ -0.333333333333333 -0.0566406250000000 -0.0566406250000000   0.666666666666667]


In [6]:
def _flatten_matrix(mat, doubled=False):
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def _unflatten_matrix(ls, dim, doubled=False):
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

def _round(value, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    if method==0:
        return QQ(round(value*denom)/denom)
    else:
        from sage.rings.continued_fraction import continued_fraction
        cf = continued_fraction(value)
        for ii, xx in enumerate(cf.quotients()):
            if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
                if ii>0:
                    return cf.convergent(ii-1)
                return 0
        return cf.value()

def _round_list(ls, force_pos=False, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    if force_pos:
        return [max(_round(xx, method, quotient_bound, denom_bound, denom), 0) for xx in ls]
    else:
        return [_round(xx, method, quotient_bound, denom_bound, denom) for xx in ls]

def _round_matrix(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    return matrix(QQ, [_round_list(xx, False, method, quotient_bound, denom_bound, denom) for xx in mat])

def _round_ldl(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, _round_matrix(mat_ldl[1]), method, quotient_bound, denom_bound, denom)
    D = diagonal_matrix(QQ, _round_list(mat_ldl[2].diagonal(), True, method, quotient_bound, denom_bound, denom))
    pl = P*L
    return pl*D*pl.T

def _round_adaptive(ls, onevec, denom=1024):
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([_round(xx, quotient_bound=resol1, denom_bound=resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    if best_vec==None:
        rvec = vector(QQ, _round_list(ls, True, method=0, denom=denom))
        best_vec = rvec/(rvec*onevec)
    return best_vec

def _fraction_print(val, thr=20):
    if len(str(val))>thr:
        return str(val.n())+"?"
    else:
        return str(val)